In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(r'./countries/countries.db')
# SQL statement to drop table
# conn.execute("DROP TABLE album")
# commit changes and close connection
conn.commit()
# conn.close()
df = pd.read_csv(r'./countries/cities.csv',on_bad_lines='skip')
df.to_sql('cities',conn,if_exists='replace',index=False)
df = pd.read_csv(r'./countries/countries.csv',on_bad_lines='skip')
df['name']=df['country_name']
df.to_sql('countries',conn,if_exists='replace',index=False)
df = pd.read_csv(r'./countries/currencies.csv',on_bad_lines='skip')
df.to_sql('currencies',conn,if_exists='replace',index=False)
df = pd.read_csv(r'./countries/economies.csv',on_bad_lines='skip')
df.to_sql('economies',conn,if_exists='replace',index=False)
df = pd.read_csv(r'./countries/languages.csv',on_bad_lines='skip')
df.to_sql('languages',conn,if_exists='replace',index=False)
df = pd.read_csv(r'./countries/populations.csv',on_bad_lines='skip')
df.to_sql('populations',conn,if_exists='replace',index=False)
df = pd.read_csv(r'./countries/eu_countries.csv',on_bad_lines='skip')
df.to_sql('eu_countries',conn,if_exists='replace',index=False)
df = pd.read_csv(r'./countries/economies2015.csv',on_bad_lines='skip')
df.to_sql('economies2015',conn,if_exists='replace',index=False)
df = pd.read_csv(r'./countries/economies2019.csv',on_bad_lines='skip')
df.to_sql('economies2019',conn,if_exists='replace',index=False)

151

In [14]:
def rsq(sql):
    return pd.read_sql_query(sql,conn)

In [15]:
rsq("select * from cities").head()

,name,country_code,city_proper_pop,metroarea_pop,urbanarea_pop
0,Abidjan,CIV,4765000,NaN,4765000
1,Abu Dhabi,ARE,1145000,NaN,1145000
2,Abuja,NGA,1235880,6000000.0,1235880
3,Accra,GHA,2070463,4010054.0,2070463
4,Addis Ababa,ETH,3103673,4567857.0,3103673


In [16]:
rsq("""SELECT * 
FROM cities
-- Inner join to countries
INNER JOIN countries
-- Match on country codes
ON cities.country_code = countries.code;""").head()

,name,country_code,city_proper_pop,metroarea_pop,urbanarea_pop,code,country_name,continent,region,surface_area,indep_year,local_name,gov_form,capital,cap_long,cap_lat,name
0,Abidjan,CIV,4765000,NaN,4765000,CIV,Cote d'Ivoire,Africa,Western Africa,322463.0,1960.0,Cote dIvoire,Republic,Yamoussoukro,-4.03050,5.33200,Cote d'Ivoire
1,Abu Dhabi,ARE,1145000,NaN,1145000,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971.0,Al-Imarat al-´Arabiya al-Muttahida,Emirate Federation,Abu Dhabi,54.37050,24.47640,United Arab Emirates
2,Abuja,NGA,1235880,6000000.0,1235880,NGA,Nigeria,Africa,Western Africa,923768.0,1960.0,Nigeria,Federal Republic,Abuja,7.48906,9.05804,Nigeria
3,Accra,GHA,2070463,4010054.0,2070463,GHA,Ghana,Africa,Western Africa,238533.0,1957.0,Ghana,Republic,Accra,-0.20795,5.57045,Ghana
4,Addis Ababa,ETH,3103673,4567857.0,3103673,ETH,Ethiopia,Africa,Eastern Africa,1104300.0,-1000.0,YeItyop´iya,Republic,Addis Ababa,38.74680,9.02274,Ethiopia


In [17]:
rsq("""-- Select name fields (with alias) and region 
SELECT cities.name AS city, countries.name AS country, countries.region 
FROM cities
INNER JOIN countries
ON cities.country_code = countries.code;""").head()

,city,country,region
0,Abidjan,Cote d'Ivoire,Western Africa
1,Abu Dhabi,United Arab Emirates,Middle East
2,Abuja,Nigeria,Western Africa
3,Accra,Ghana,Western Africa
4,Addis Ababa,Ethiopia,Eastern Africa


In [18]:
rsq("""SELECT c.code AS country_code, c.name, e.year, e.inflation_rate
FROM countries AS c
INNER JOIN economies AS e ON c.code = e.code;""").head()

,country_code,name,year,inflation_rate
0,AFG,Afghanistan,2010,2.179
1,AFG,Afghanistan,2015,-1.549
2,NLD,Netherlands,2010,0.932
3,NLD,Netherlands,2015,0.220
4,ALB,Albania,2010,3.605


In [19]:
rsq("""SELECT c.name AS country, l.name AS language, official
FROM countries AS c
INNER JOIN languages AS l
-- Match using the code column
USING (code)""").head()

,country,language,official
0,Afghanistan,Dari,1
1,Afghanistan,Other,0
2,Afghanistan,Pashto,1
3,Afghanistan,Turkic,0
4,Netherlands,Dutch,1


In [20]:
rsq("""SELECT c.name AS country, l.name AS language 
FROM countries AS c 
JOIN languages AS l USING (code);""").head()

,country,language
0,Afghanistan,Dari
1,Afghanistan,Other
2,Afghanistan,Pashto
3,Afghanistan,Turkic
4,Netherlands,Dutch


In [21]:
rsq("""-- Rearrange SELECT statement, keeping aliases
SELECT c.name AS country, l.name AS language
FROM countries AS c
INNER JOIN languages AS l
USING(code)
-- Order the results by language
ORDER BY language""").head()

,country,language
0,Djibouti,Afar
1,Eritrea,Afar
2,Ethiopia,Afar
3,South Africa,Afrikaans
4,Namibia,Afrikaans


In [22]:
rsq("""-- Rearrange SELECT statement, keeping aliases
SELECT c.name AS country, l.name AS language
FROM countries AS c
INNER JOIN languages AS l
USING(code)
-- Order the results by language
ORDER BY language""").head()

,country,language
0,Djibouti,Afar
1,Eritrea,Afar
2,Ethiopia,Afar
3,South Africa,Afrikaans
4,Namibia,Afrikaans


In [23]:
rsq("""-- Select relevant fields
-- Inner join countries and populations, aliased, on code
SELECT name, year, fertility_rate 
FROM countries AS c 
INNER JOIN populations AS p 
ON c.code = p.country_code;""").head()

,name,year,fertility_rate
0,Afghanistan,2010,5.746
1,Afghanistan,2015,4.653
2,Netherlands,2010,1.790
3,Netherlands,2015,1.710
4,Albania,2010,1.663


In [24]:
rsq("""-- Select fields
SELECT name, e.year, fertility_rate, unemployment_rate
FROM countries AS c
INNER JOIN populations AS p
ON c.code = p.country_code
-- Join to economies (as e)
INNER JOIN economies AS e
-- Match on country code
ON c.code = e.code;""").head()

,name,year,fertility_rate,unemployment_rate
0,Afghanistan,2010,4.653,NaN
1,Afghanistan,2015,4.653,NaN
2,Afghanistan,2010,5.746,NaN
3,Afghanistan,2015,5.746,NaN
4,Netherlands,2010,1.710,4.995


In [25]:
rsq("""SELECT name, e.year, fertility_rate, unemployment_rate
FROM countries AS c
INNER JOIN populations AS p
ON c.code = p.country_code
INNER JOIN economies AS e
ON c.code = e.code
-- Add an additional joining condition such that you are also joining on year
	AND 
	p.year = e.year;""").head()

,name,year,fertility_rate,unemployment_rate
0,Afghanistan,2010,5.746,NaN
1,Afghanistan,2015,4.653,NaN
2,Netherlands,2010,1.790,4.995
3,Netherlands,2015,1.710,6.891
4,Albania,2010,1.663,14.000
